In [9]:
import numpy as np
import pandas as pd
from polyphys.probe.prober import fsd, stamps_report
from polyphys.manage.organizer import sort_filenames
from polyphys.analyze import correlations
from polyphys.analyze import clusters
from polyphys.visualize.plotter import ATTRIBUTE_LABELS as AL
from polyphys.visualize.plotter import SIZE_MEASURES_LABELS as SML
from polyphys.visualize.plotter import DUTCH_FEILD_COLORS as DF_colors
from polyphys.manage.parser import HnsCyl, HnsCub, SumRuleCyl
from typing import Dict, List, Tuple, Optional, Union
from itertools import combinations
from collections import defaultdict

import MDAnalysis as mda
from MDAnalysis.analysis.base import AnalysisFromFunction
from MDAnalysis import transformations as mda_trans
from MDAnalysis.analysis import distances as mda_dist
import statsmodels.tsa.stattools as tsas

import imageio

from polyphys.visualize.plotter import PROJECT_DETAILS as PD
from polyphys.visualize.plotter import TITLE_STYLES as TS

from matplotlib.image import NonUniformImage
from matplotlib.patches import Circle, Rectangle
from matplotlib import cm

from glob import glob
import seaborn as sns
import matplotlib.pyplot as plt

paul_tol_bright = ['#4477AA', '#EE6677', '#228833', '#CCBB44', '#66CCEE',
                    '#AA3377', '#BBBBBB', '#000000']

import scipy
def fit_exponential_decay(x, y):
    r"""Fit a function to an exponential decay

    .. math::  y = \exp\left(- \frac{x}{a}\right)

    Parameters
    ----------
    x, y : array_like
      The two arrays of data

    Returns
    -------
    a : float
      The coefficient *a* for this decay

    Notes
    -----
    This function assumes that data starts at 1.0 and decays to 0.0

    """
    def exp_func(x, a):
        return np.exp(-1*x/a)
    a = scipy.optimize.curve_fit(exp_func, x, y)[0][0]
    return a
def fit_exponential_growth(x, y):
    r"""Fit a function to an exponential decay

    .. math::  y = \exp\left(- \frac{x}{a}\right)

    Parameters
    ----------
    x, y : array_like
      The two arrays of data

    Returns
    -------
    a : float
      The coefficient *a* for this decay

    Notes
    -----
    This function assumes that data starts at 1.0 and decays to 0.0

    """
    def exp_func(x, a):
        return np.exp(x/a)
    a = scipy.optimize.curve_fit(exp_func, x, y)[0][0]
    return a


# Sum-rule: single whole folder

In [ ]:
database = '/Users/amirhsi_mini/research_data/sumrule/'
hierarchy = 'N*'
group = 'bug'
files = glob(database + hierarchy)
sim_name = SumRuleCyl(files[0], 'whole', 'cylindrical', group, 'linear')
print(sim_name.whole)
pattern = database + sim_name.whole + '-' + group
gyr_t = np.load(pattern + '-gyrTMon.npy')
time = 5000 * np.arange(0,len(gyr_t))
fsd_t = np.load(pattern + '-fsdTMon.npy')
flory_t = np.load(pattern + '-floryTMon.npy')
asphere = np.load(pattern + '-asphericityTMon.npy')
shape_t = np.load(pattern + '-shapeTMon.npy')
trans_size_t = np.load(pattern + '-transSizeTMon.npy')

In [7]:
project= 'SumRuleCyl'
allInOne_db = '/Users/amirhsi_mini/research_data/sumrule/analysis/'
space_dbs = glob(allInOne_db + PD[project]['space_pat'])
space_dbs

['/Users/amirhsi_mini/research_data/sumrule/analysis/N2000D60.0ac4.0-whole-equilProps.csv',
 '/Users/amirhsi_mini/research_data/sumrule/analysis/N2000D60.0ac4.0-bug-ens',
 '/Users/amirhsi_mini/research_data/sumrule/analysis/N2000D60.0ac4.0-bug-chainSize-acf.parquet.brotli',
 '/Users/amirhsi_mini/research_data/sumrule/analysis/N2000D60.0ac4.0-bug-ensAvg',
 '/Users/amirhsi_mini/research_data/sumrule/analysis/N2000D60.0ac4.0-bug-chainSize.parquet.brotli']

In [10]:
# read data
acf = []
acf_space_dbs = [space_db for space_db in space_dbs if
                 space_db.endswith('acf.parquet.brotli')]
# Define lag times
dumping_freq = {'TransFociCyl': 'bdump', 'TransFociCub': 'bdump',
                'SumRuleCyl': 'bdump', 'HnsCub': 'ndump', 'HnsCyl': 'ndump'
                }

for space_db in acf_space_dbs:
    space_acf = pd.read_parquet(space_db)
    ensembles = space_acf['ensemble_long'].unique()
    space_acf.rename(columns={'time':'time_dump_dt'}, inplace=True)
    
    space_acf['lag_time'] = 0
    space_acf['lag_index'] = 0
    space_acf['space_title'] = ''
    space_acf['space_title_short'] = ''
    space_acf['universe'] = ''
    
    for ens in ensembles:
        s_info = PD[project]['parser'](ens, 'ensemble_long', PD[project]['geometry'], PD[project]['group'], PD[project]['topology'], ispath=False)
        if project not in TS.keys():
            raise ValueError(f"Project: '{project}' is not defined.")

        style = TS[project]
        cond = space_acf['ensemble_long']==ens
        space_acf.loc[cond,'space_title'] = style['title'](s_info)
        space_acf.loc[cond,'space_title_short'] = style['short'](s_info)
        space_acf.loc[cond,'universe'] = style['universe'](s_info)
        
        n_lags = len(space_acf.loc[cond,'time_dump_dt'])
        bdump = getattr(s_info, dumping_freq[project])
        space_acf.loc[cond,'lag_time'] = space_acf.loc[cond,'time_dump_dt'] * bdump
        space_acf.loc[cond,'lag_index'] = np.arange(0,n_lags,dtype=np.int_)
    
    acf.append(space_acf)

# Concatenate and reset DataFrame
acf = pd.concat(acf, axis=0, ignore_index=True)
acf.reset_index(inplace=True, drop=True)

# Obtain and sort unique spaces
spaces = acf['space'].drop_duplicates().sort_values()
spaces = sorted(spaces, key=organizer.sort_by_alphanumeric)

# Handle rounding of specific projects
if project == 'HnsCyl':
    replacements = {0.09: 0.08, 0.15: 0.16, 0.21: 0.2, 0.31: 0.32}
    acf['phi_c_bulk_round'].replace(replacements, inplace=True)
    acf = acf.loc[~acf['phi_c_bulk_round'].isin([0.06, 0.18]), :]
elif project == 'TransFociCyl':
    acf = acf.loc[~acf['phi_c_bulk_round'].isin([0.025, 0.05, 0.075, 0.125, 0.175]), :]

print('Unique phi_c_bulk:', sorted(acf['phi_c_bulk_round'].unique()))

# Reset index and drop unnecessary columns
acf.reset_index(inplace=True, drop=True)
cols_to_drop = [col for col in acf.columns if col.endswith(('sem', 'var')) or col == 'phi_c_bulk']
acf.drop(columns=cols_to_drop, inplace=True)

# Define list of unique physical properties and rename columns
ext = '-acf-mean'
properties = sorted(prop.split(ext)[0] for prop in acf.columns if ext in prop)
acf.columns = acf.columns.str.replace(ext, '').str.replace('-mean', '')

print(acf.columns)

# Re-sort unique spaces
spaces = acf['space'].drop_duplicates().sort_values()
spaces = sorted(spaces, key=organizer.sort_by_alphanumeric)

/Users/amirhsi_mini/OneDrive - University of Waterloo/PhD Research/Jupyter/PolyPhys/polyphys/manage/parser.py:672: UserWarning: It is assumed that 'nc' is the last attribute short-key in a lineage_name of types: 'ensemble', 'ensemble_long', 'whole', 'segment'.
  warnings.warn(convention_warning, UserWarning)


NameError: name 'organizer' is not defined

# HNS-DNA project: Chain minimization stage

## Lammps-generated $R_g$

In [ ]:
path_txt = '/Users/amirhsi_mini/research_data/N1000_tests/N1000*/N*gyrTDna.txt'
paths =  sorted(glob(path_txt))
gyr_t_list = []
for path in paths:
    sim_info = HnsCyl(
        path,
        'segment',
        'cylindrical',
        'nucleoid',
        'ring'
    )
    df = pd.read_csv(path, sep=' ', names=['time', 'gyr'], index_col=False, skiprows=[0])
    for prop_ in sim_info.attributes:
        df[prop_] = getattr(sim_info, prop_)
    df['phi_c_bulk_round'] = df['phi_c_bulk'].round(2)
    gyr_t_list.append(df)
gyr_t = pd.concat(gyr_t_list)
# First visualize the whole set and realized that for timestep>=1.25*10^8, the chain
# is in equilibrium, so
time_cond = gyr_t['time'] >= 1.25*10**8
gyr_t_equil = gyr_t.loc[time_cond]

In [ ]:
gyr_t_equil.groupby(['nhns','dcrowd','phi_c_bulk_round','ensemble_id'])['gyr'].mean()

In [ ]:
nmon = 1000
dcyl = 15
data = gyr_t_equil
hue_attr = 'phi_c_bulk_round'
hue_order = sorted(data[hue_attr].unique())
style_attr = 'dcrowd'
size_attr = 'nhns'
color_palette = paul_tol_bright[:len(hue_order)]

# Plotting loop length statistics
height = 4
aspect = 1.618 # Golden ratio
font_family = 'sans-serif' #'Times New Roman' #'Times New Roman' #'sans-serif'#'DejaVu Sans'#'sans-serif' # 
math_font = 'dejavusans' #'dejavusans' #'dejavusans' #'dejavuserif' #'stixsans'
font_scale = 1.5
plot_context = 'paper'
save_to = './'
ext = 'pdf'
rc_params = {
    'mathtext.fontset': math_font, #'regular',
    #'mathtext.default': 'regular',
    'axes.formatter.use_mathtext': True,
    "font.family": font_family,  # use serif/main font for text elements
}
legend_kws = {
    'ncol': 1,
    'frameon': True,
    'facecolor': 'white',
    'framealpha': 1.0,
    'edgecolor': 'black',
    'columnspacing': 1,
    'borderpad': 0.2,
    'handletextpad': 0.1,
    'markerscale': 3
}
facet_kws = {
    'sharey': False,
    'sharex': 'col',
    'legend_out': True
}
sns.set_theme(
    context=plot_context,
    style='ticks',
    palette=color_palette,
    font=font_family,
    font_scale=font_scale,
    rc=rc_params
)
fig, ax = plt.subplots(1,1, figsize=(height*aspect, height))
f_grid = sns.lineplot(
    ax=ax,
    data=data,
    x='time',
    y='gyr',
    hue=hue_attr,
    hue_order=hue_order,
    style=style_attr,
    markers=False,
    size=size_attr,
    palette=color_palette,
    legend='full',
    alpha=0.5,
    lw=1,
    )
#slicer = 0
#gyr_mean = np.mean(gyr_t[slicer:])
#ax.axhline(y=gyr_mean, c='red', label=fr'$\langle R_g\rangle={round(gyr_mean, 3)}$')
#ax.axvline(x=time[slicer], c='gray',ls='--', label=fr'$t/\tau_{{LJ}}={time[slicer]}$')
f_grid.set_ylabel(r'$R_g(t)$')

f_grid.set_title(fr'Cylindrical confinement: $N={nmon}, D/a_m={dcyl}$')

ax.legend(loc='upper left', bbox_to_anchor=(1,1), frameon=False)

plt.show()
fig.savefig('HnsCyl-chainsize_dna_minimize.pdf')

 ## Probe stage - nucloeid or bug only:

In [ ]:
%%time
## 14 min for 2 trjs
## 9 min for N=200 no crowd no hns with all transformations, ndump=2000
## 9 min for N=200 no crowd no hns no need for all transformations, ndump=5000
sliced = False
slice_id = 125000000
save_to = "./"
path_txt = '/Users/amirhsi_mini/research_data/N1000_tests/N1000*/N*'
pairs =  sorted(glob(path_txt))
pairs = sort_filenames(pairs, ['j03.ring.dnaMinimize.nucleoid.lammpstrj', 'nucleoid.data'])
for (trj, topo) in pairs:
    sim_info= HnsCyl(
        trj,
        'segment',
        'cylindrical',
        'nucleoid',
        'ring'
    )
    dt = 1
    ndump = 20000
    cell = mda.Universe(
        topo,
        trj,
        topology_format='DATA',
        format='LAMMPSDUMP',
        lammps_coordinate_convention='unscaled',
        unwrap_images=False,
        atom_style="id resid type x y z", 
        dt = ndump*dt # timestep of the first configuration
    )
    sim_name = sim_info.lineage_name + "-" + sim_info.group
    print(sim_name)
    if sliced is True:
        sliced_trj = cell.trajectory[:slice_id]
    else:
        sliced_trj = cell.trajectory
    n_frames = len(sliced_trj)
    bug = cell.select_atoms('resid 1')  # the bug
    # defining collectors
    # bug:
    gyr_t = []
    fsd_t = []
    # distance matrices
    for _ in sliced_trj:
        # bug:
        # various measures of chain size
        gyr_t.append(bug.radius_of_gyration())
        fsd_t.append(fsd(bug.positions))
    np.save(save_to + sim_name + '-fsdTMon.npy', np.array(fsd_t))
    np.save(save_to + sim_name + '-gyrTMon.npy', np.array(gyr_t))
    print('done.')

### Visualze $L$ and $R_g$ from probing bug/nucleoid

In [ ]:
path_txt = './N*.npy'
paths =  sorted(glob(path_txt))
variables = sorted(np.unique([path.split('.')[-2] for path in paths]))
variables = [var.split('-')[-1] for var in variables]
print(variables)
all_var_list = []
ndump_minimized = 20000
first_time_step = 200200000
for var in ['fsdTMon', 'gyrTMon']:
    path_txt = f'./N*-{var}.npy'
    paths =  sorted(glob(path_txt))
    var_t_list = []
    for path in paths:
        sim_info = HnsCyl(
            path,
            'segment',
            'cylindrical',
            'nucleoid',
            'ring'
        )
        var_dict = {var: np.load(path)}
        df = pd.DataFrame.from_dict(var_dict)
        for prop_ in sim_info.attributes:
            df[prop_] = getattr(sim_info, prop_)
        df['phi_c_bulk_round'] = df['phi_c_bulk'].round(2)
        df['timestep'] = df.index * ndump_minimized +  first_time_step
        var_t_list.append(df)
    var_t = pd.concat(var_t_list)
    all_var_list.append(var_t)
# First visualize the whole set and realized that for timestep>=1.25*10^8, the chain
# is in equilibrium, so
all_var = pd.concat(all_var_list, axis=1)
all_var = all_var.loc[:, ~all_var.columns.duplicated()]
#time_cond = gyr_t['time'] >= 1.25*10**8
#gyr_t_equil = gyr_t.loc[time_cond]
cols_to_drop = ['nmon', 'bend_mm', 'dcyl', 'ncrowd',
       'lcyl', 'ensemble_id', 'segment_id', 'dmon', 'mmon', 'dhns', 'mhns',
       'mcrowd', 'eps_others', 'phi_m_bulk', 'rho_m_bulk', 'phi_c_bulk',
       'rho_c_bulk', 'phi_hns_bulk', 'rho_hns_bulk', 'eps_hm', 'dt', 'ndump',
       'adump']
all_var.drop(columns=cols_to_drop, inplace=True)
id_vars = ['nhns', 'dcrowd', 'phi_c_bulk_round', 'timestep']
value_vars = ['fsdTMon', 'gyrTMon']
all_var_melt = pd.melt(all_var, id_vars=id_vars, value_vars=value_vars, value_name='chainsize_val', var_name='chainsize')


In [ ]:
all_var.loc[all_var['timestep']>=first_time_step+1000*ndump_minimized].groupby(['nhns', 'dcrowd', 'phi_c_bulk_round'])[value_vars].agg(np.mean)

In [ ]:
dcyl = 15
nmon = 1000

plot_params = {
    'paper': {
    'height': 3,
    'aspect': 1.618, #1 * 16/9, # 1.5 * 1.618 # Golden ratio
    'font_scale': 1.5,
    'lw': 1.75,
    'bbox_to_anchor':(0.65, 0.33),
    'bbox_to_anchor_fig':(0.47, 0.32)
    }
}
fig_title_kws = {'y': 0.96, 'x': 0.45}
save_to = './'
ext = 'pdf'
font_family = 'sans-serif'
math_font = 'dejavusans'
rc_params= {
    'mathtext.fontset': math_font,
    'axes.formatter.use_mathtext': True,
}
facet_kws = {
    'sharey': 'row',
    'sharex': 'col',
    'legend_out': True
}

legend_kws = {
    'frameon': False,
    'facecolor': 'white',
    'framealpha': 1.0,
    'columnspacing': 1,
    'borderpad': 0.2,
    'handletextpad': 0.2,
    'edgecolor': 'black',
    'markerscale': 2
}

data = all_var = all_var_melt.loc[all_var_melt['timestep']>=first_time_step+1000*ndump_minimized]

x_prop = 'timestep'
y_prop = 'chainsize_val'

row_attr = 'chainsize'
row_order = value_vars
col_attr = 'nhns'
col_order = sorted(data[col_attr].unique())

hue_attr = 'phi_c_bulk_round'
hue_order = sorted(data[hue_attr].unique())
style_attr = 'dcrowd'
#size_attr = 'nhns'
color_palette = paul_tol_bright[:len(hue_order)]

plot_context = 'paper'
sns.reset_defaults()
with sns.axes_style('ticks', rc=rc_params):
    with sns.plotting_context(
        plot_context,
        font_scale=plot_params[plot_context]['font_scale']
        ):
        grid = sns.relplot(
            data=data,
            x=x_prop,
            y=y_prop,
            col=col_attr,
            col_order=col_order,
            row=row_attr,
            style=style_attr,
            markers=False,
            dashes=True,
            hue=hue_attr,
            hue_order=hue_order,
            kind='line',
            height=plot_params[plot_context]['height'],
            aspect=plot_params[plot_context]['aspect'],
            legend='full',
            palette=color_palette,
            facet_kws=facet_kws,
            errorbar=None,
            alpha=0.7,
            lw=plot_params[plot_context]['lw']
            )
        grid.set_titles(AL[col_attr] + r"$={col_name}$")
        grid.set_xlabels('timestep')
        grid.fig.suptitle(fr"$N={nmon}, D/a_m={dcyl}$")
        for ax, row in zip(grid.axes[:,0], row_order):
            print(ax, row)
            print(SML[row]['symbol'])
            ax.set_ylabel(SML[row]['symbol'])
        grid.tight_layout(w_pad=0.35, h_pad=0.25)
        output = "-".join(
            ["dna_minimized", f"nmon{nmon}", f"dcyl{dcyl}"])
        plt.show()
        grid.savefig(save_to + output + '.' + ext, bbox_inches='tight')


## creating data files from equilibrated single chain trajectories:

In [ ]:
%%time
sliced = False
slice_id = 2500
ndump = 20000
first_timestep = 200200000

save_to = "./"
path_txt = '/Users/amirhsi_mini/research_data/N1000_tests/N1000*/N*'
pairs =  sorted(glob(path_txt))
pairs = sort_filenames(pairs, ['j03.ring.dnaMinimize.nucleoid.lammpstrj', 'j03.ring.dnaMinimize.nucleoid.data'])
for (trj, topo) in pairs[:1]:
    print(trj)
    print(topo)
    sim_info= HnsCyl(
        trj,
        'segment',
        'cylindrical',
        'nucleoid',
        'ring'
    )
    cell = mda.Universe(
        topo,
        trj,
        topology_format='DATA',
        format='LAMMPSDUMP',
        lammps_coordinate_convention='unscaled',
        unwrap_images=False,
        atom_style="id resid type x y z",
    )
    
    bug = cell.select_atoms('resid 1')  # the bug
    workflow = [
            mda_trans.unwrap(cell.atoms),
            mda_trans.center_in_box(bug),
            mda_trans.wrap(cell.atoms),
            
        ]
    cell.trajectory.add_transformations(*workflow)
    sim_name = sim_info.lineage_name
    if sliced is True:
        sliced_trj = cell.trajectory[:slice_id]
    else:
        sliced_trj = cell.trajectory
    #for ts in sliced_trj[0::10]:
    for ts in sliced_trj:
        print(bug.radius_of_gyration())
        data_name = sim_name + f".timestep{ndump * ts.frame +first_timestep}.data"
        with mda.Writer(data_name) as W:
            W.write(bug.atoms)
    print('done.')

## Visulziing: all or nucleoid or bug files

In [ ]:
for var in variables:
    print(var)
    if var not in ['distMatTMonHnsPatch.npy', 'principalTMon.npy', 'stamps.csv']:
        dist_mon_hpatch_t = np.load('/Users/amirhsi_mini/research_data/do_not_delete/N200kbmm2r4.5nh12ac2lz75nc386ens4.ring-nucleoid-'+var)
        print(dist_mon_hpatch_t.shape)
        plt.plot(dist_mon_hpatch_t)
        if var not in ['bondCosineCorrVecMon.npy', 'loopLengthMonHist.npy']:
            plt.axhline(np.mean(dist_mon_hpatch_t), c='red')
        plt.savefig('N200kbmm2r4.5nh12ac2lz75nc386ens4.ring-' + var[:-3]+'pdf')
        plt.close()

In [ ]:
gyr_t = np.load('/Users/amirhsi_mini/research_data/hns_cyl-trjs/N200kbmm2r4.5nh12ac2lz75nc552ens4.ring-nucleoid-gyrTMon.npy')
time = 5000 * np.arange(0,len(gyr_t))
fsd_t = np.load('/Users/amirhsi_mini/research_data/hns_cyl-trjs/N200kbmm2r4.5nh12ac2lz75nc552ens4.ring-nucleoid-fsdTMon.npy')
asphere = np.load('/Users/amirhsi_mini/research_data/hns_cyl-trjs/N200kbmm2r4.5nh12ac2lz75nc552ens4.ring-nucleoid-asphericityTMon.npy')
shape_t = np.load('/Users/amirhsi_mini/research_data/hns_cyl-trjs/N200kbmm2r4.5nh12ac2lz75nc552ens4.ring-nucleoid-shapeTMon.npy')
trans_size_t = np.load('/Users/amirhsi_mini/research_data/hns_cyl-trjs/N200kbmm2r4.5nh12ac2lz75nc552ens4.ring-nucleoid-transSizeTMon.npy')

In [ ]:
color_palette = 'tab10'
plot_context = 'paper'
font_scale = 1.5
height = 6
aspect = 1.618
ylabel_pad = 50
rc_params= {
    #'axes.facecolor': 'aliceblue',
    'mathtext.default': 'regular',
    'text.usetex': True,
    #'axes.grid': True,
    #'axes.grid.axis': 'both',
    #'axes.grid.which': 'both'
}
fig, axes = plt.subplots(nrows=5, ncols=1,figsize=(aspect*height, height), sharex=True)
sns.set_theme(
    context=plot_context,
    style='ticks',
    palette=color_palette,
    font='Times New Roman',
    font_scale=font_scale,
    rc=rc_params
)
slicer = 0
ax = axes[0]
ax.plot(time, gyr_t)
gyr_mean = np.mean(gyr_t[slicer:])
ax.axhline(y=gyr_mean, c='red', label=fr'$\langle R_g\rangle={round(gyr_mean, 3)}$')
ax.axvline(x=time[slicer], c='gray',ls='--', label=fr'$t/\tau_{{LJ}}={time[slicer]}$')
ax.set_ylabel(r'$R_g(t)$')
ax.set_title(r'Cylindrical confinement: $N=200, D/a_m=8, a_c/a_m=2, \phi_c=0.4, n_{{hns}}=12$')
ax.legend(ncols=2)

ax = axes[1]
ax.plot(time, fsd_t)
fsd_mean = np.mean(fsd_t[slicer:])
ax.axhline(y=fsd_mean, c='red', label=fr'$\langle L\rangle={np.format_float_positional(fsd_mean,precision=3)}$')
ax.axvline(x=time[slicer], c='gray',ls='--', label=fr'$t/\tau_{{LJ}}={time[slicer]}$')
ax.set_ylabel(r'$L$')
ax.set_xlabel(r'$t/\tau_{LJ}$')
ax.legend(ncols=2)

ax = axes[2]
ax.plot(time, trans_size_t)
fsd_mean = np.mean(trans_size_t[slicer:])
ax.axhline(y=fsd_mean, c='red', label=fr'$\langle R_{{\perp}}\rangle={round(fsd_mean,3)}$')
ax.axvline(x=time[slicer], c='gray',ls='--', label=fr'$t/\tau_{{LJ}}={time[slicer]}$')
ax.set_ylabel(r'$R_{\perp}$')
ax.set_xlabel(r'$t/\tau_{LJ}$')
ax.legend(ncols=2)

ax = axes[3]
ax.plot(time, asphere)
fsd_mean = np.mean(asphere[slicer:])
ax.axhline(y=fsd_mean, c='red', label=fr'$\langle \Delta\rangle={round(fsd_mean,3)}$')
ax.axvline(x=time[slicer], c='gray',ls='--', label=fr'$t/\tau_{{LJ}}={time[slicer]}$')
ax.set_ylabel(r'$\Delta$')
ax.set_xlabel(r'$t/\tau_{LJ}$')
ax.legend(ncols=2)

ax = axes[4]
ax.plot(time, shape_t)
fsd_mean = np.mean(shape_t[slicer:])
ax.axhline(y=fsd_mean, c='red', label=fr'$\langle S\rangle={round(fsd_mean,3)}$')
ax.axvline(x=time[slicer], c='gray',ls='--', label=fr'$t/\tau_{{LJ}}={time[slicer]}$')
ax.set_ylabel(r'$S$')
ax.set_xlabel(r'$t/\tau_{LJ}$')
ax.legend(ncols=2)

plt.show()
fig.savefig('chainsize.pdf')

In [ ]:
bond_lengths = np.load('/Users/amirhsi_mini/research_data/hns_cyl-trjs/N200kbmm2r4.5nh12ac2lz75nc552ens4.ring-nucleoid-bondLengthVecMon.npy')
cosine_corrs = np.load('/Users/amirhsi_mini/research_data/hns_cyl-trjs/N200kbmm2r4.5nh12ac2lz75nc552ens4.ring-nucleoid-bondCosineCorrVecMon.npy')

#trj_cosine_corrs = np.mean(trj_cosine_corrs_mean,axis=0)
mean_bond = np.mean(bond_lengths[0])
print('bond length: ', mean_bond)
bond_lags = np.arange(1, len(cosine_corrs)+1)
per_to_bond = fit_exponential_decay(
    bond_lags,
    cosine_corrs
)
print('persisten to bond ratio', per_to_bond)

In [ ]:
color_palette = 'tab10'
#color_palette = ['orange', 'darkgoldenrod', 'forestgreen']
save_to = './'
ext = 'pdf'
plot_context = 'paper'
font_scale = 1.5
height = 4
aspect = 1.5*1.618
ylabel_pad = 50
rc_params= {
    #'axes.facecolor': 'aliceblue',
    'mathtext.default': 'regular',
    'text.usetex': True,
    #'axes.grid': True,
    #'axes.grid.axis': 'both',
    #'axes.grid.which': 'both'
}
facet_kws = {
    'sharey': False,
    'sharex': False,
    'legend_out': True,
}
fig_title_kws = {'fontsize': 34, 'x': 0.5, 'y': 0.93}
loc='lower left'
font_family = 'sans-serif'
move_legend_kws = {
    #'ncol': 1,
    #'bbox_to_anchor': (1., 0.95),
    'frameon': False,
    #'facecolor': 'aliceblue',
    #'borderpad': 0.1,
    'markerscale': 1
}
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(aspect*height, height))
sns.set_theme(
    context=plot_context,
    style='ticks',
    palette=color_palette,
    font='Times New Roman',
    font_scale=font_scale,
    rc=rc_params
)

ax.scatter(bond_lags,cosine_corrs,c='green',marker='o',s=10,alpha=0.7)
ax.plot(bond_lags,np.exp(-(bond_lags-1)/per_to_bond),color='red',label=fr'$\exp[-j/(l_p/l_b)]$ with $l_p/l_b={round(per_to_bond,3)}$ (fitted to $20$ closest bonds)')
#short_mean = np.mean(long[55000:,1])
ax.axhline(y=0,c='black', lw=0.75)
#ax.axvline(x=0.005*long[55000,0],c='gray',ls='--',label='Equilibrium reached!')
ax.legend()
#ax.set_ylabel(r'$R_g(t)$')
ax.set_xlabel('Bond index lag, j')
#bond_grid.set_ylabels(r'$\langle\cos(\theta_{i,i+j})\rangle$')
ax.set_ylabel(r'$C(j)=\langle\hat{l}_i.\hat{l}_{i+j}\rangle$')
#ax.text(120, 0.7, fr'fit to first $25$ bonds')
ax.set_title(r'Cylindrical confinement: $N=200, D/a_m=8, a_c/a_m=2, \phi_c=0.4, n_{{hns}}=12$')
plt.show()
fig.savefig('bond_correlations.pdf')

In [ ]:
# confined
gyr1 = np.loadtxt('/Users/amirhsi_mini/research_data/test_hns_cyl/N1000kbmm24nh0ac1r8lz675nc0ens1.ring/N1000kbmm24nh0ac1r8lz675nc0ens1.ring.gyrTDna.txt')
# free
gyr2 = np.loadtxt('/Users/amirhsi_mini/research_data/test_hns_cyl/N1000kbmm24nh0ac1l250nc0ens1.ring/N1000kbmm24nh0ac1l250nc0ens1.ring.gyrTDna.txt')
#gyr1 = np.loadtxt('./N1000dt0.005ndump2000adump5000kbmm25r7.5lz600nh0ac1nc0ens1.gyrTDna.txt')
#gyr2 = np.loadtxt('./N1000epshm29kbmm25nh0ac1nc0l500dt0.005ndump2000adump5000ens1.gyrTDna.1.txt')

In [ ]:
color_palette = 'tab10'
#color_palette = ['orange', 'darkgoldenrod', 'forestgreen']
plot_context = 'paper'
font_scale = 1.5
height = 5
aspect = 1.618
ylabel_pad = 50
rc_params= {
    #'axes.facecolor': 'aliceblue',
    'mathtext.default': 'regular',
    'text.usetex': True,
    #'axes.grid': True,
    #'axes.grid.axis': 'both',
    #'axes.grid.which': 'both'
}

fig, ax = plt.subplots(nrows=2, ncols=1,figsize=(aspect*height, height), sharex=True)
sns.set_theme(
    context=plot_context,
    style='ticks',
    palette=color_palette,
    font='Times New Roman',
    font_scale=font_scale,
    rc=rc_params
)
slice_id = 1000
ax.plot(0.005*gyr1[:,0],gyr1[:,1])
gyr1_mean = np.mean(gyr1[slice_id:,1])
ax.set_title(r'Cylindrical confinement: $N=1000, D/a_m=15, \phi_c=0, n_{{hns}}=0$')
ax.axhline(y=gyr1_mean,c='red',label=fr'$\langle R_g\rangle={round(gyr1_mean,3)}$')
#ax.axvline(x=0.005*gyr1[slice_id,0],c='gray',ls='--',label=fr'$t/\tau_{{LJ}}={0.005*gyr1[slice_id,0]}$')
ax.legend()

ax.plot(0.005*gyr2[:,0],gyr2[:,1])
slice_id = 1000
gyr2_mean = np.mean(gyr2[slice_id:,1])
ax.set_title(r'Free space: $N=1000, \phi_c=0, n_{{hns}}=0$')
ax.axhline(y=gyr2_mean,c='red',label=fr'$\langle R_g\rangle={round(gyr2_mean,3)}$')
#ax.axvline(x=0.005*gyr2[slice_id,0],c='gray',ls='--',label=fr'$t/\tau_{{LJ}}={0.005*gyr2[slice_id,0]}$')
ax.legend()
fig.tight_layout()
plt.show()

##  all files

In [ ]:
paths = sorted(glob('/Users/amirhsi_mini/research_data/HnsCyl-probe/*'))
paths

In [ ]:
variables = sorted(np.unique([path.split('-')[-1] for path in paths]))
print(variables)
hist_variables = [var for var in variables if 'Hist' in var]
print(hist_variables)
phi_variables = [var for var in variables if 'Phi' in var]
print(phi_variables)
edge_variables = [var for var in variables if 'Edge' in var]
edge_variables

In [ ]:
for var in hist_variables:
    if var.startswith('xyHist'):
        print(var)
        dist_mon_hpatch_t = np.load('/Users/amirhsi_mini/research_data/HnsCyl-probe/N200kbmm2r4.5nh0ac1lz75nc5760ens4--all-'+var)
        print(dist_mon_hpatch_t)
        plt.plot(dist_mon_hpatch_t)
        plt.savefig('N200kbmm2r4.5nh0ac2lz75nc111ens1.j01.ring-' + var[:-3]+'pdf')
        plt.close()

In [ ]:
hist_variables

In [ ]:
filename = 'N200kbmm2r4.5nh30ac1lz75nc5760ens4'
#
filename = 'N200kbmm2r4.5nh30ac2lz75nc552ens4'
for var in hist_variables:
    if var.startswith('rHist'):
        fig, ax = plt.subplots()
        dist_mon_hpatch_t = np.load('/Users/amirhsi_mini/research_data/HnsCyl-probe/'+filename+'-all-'+var)
        redges = np.load('/Users/amirhsi_mini/research_data/HnsCyl-probe/'+filename+'-all-'+var[0]+'Edge'+ var[-7:])
        rcenters = (redges[:-1] + redges[1:]) / 2
        ax.plot(rcenters, dist_mon_hpatch_t)
        ax.axvline(4, c='red',ls=':')
        ax.axvline(3.5, c='red',ls=':')
        ax.axvline(3, c='red',ls=':')
        ax.axvline(2.5, c='red',ls=':')
        ax.set_xlabel(r"$2r/D$")
        ax.set_ylabel(var[:-4] + r": $\rho(r)$")
        plt.savefig(filename +'-'+ var[:-3]+'pdf')
        plt.show()

In [ ]:
interp = 'bilinear'
filename = 'N200kbmm2r4.5nh30ac1lz75nc5760ens4'
#filename = 'N200kbmm2r4.5nh30ac2lz75nc552ens4'
for var in hist_variables:
    if var.startswith('xyHist'):
        fig, ax = plt.subplots()
        dist_mon_hpatch_t = np.load('/Users/amirhsi_mini/research_data/HnsCyl-probe/'+filename+'-all-'+var)
        xedges = np.load('/Users/amirhsi_mini/research_data/HnsCyl-probe/'+filename+'-all-'+var[0]+'Edge.npy')
        print(np.ptp(xedges))
        xcenters = (xedges[:-1] + xedges[1:]) / 2
        yedges = np.load('/Users/amirhsi_mini/research_data/HnsCyl-probe/'+filename+'-all-'+var[1]+'Edge.npy')
        ycenters = (yedges[:-1] + yedges[1:]) / 2
        circle = Circle((0, 0), 4.5, facecolor='none',
                    edgecolor='black', linewidth=1, alpha=1, zorder=1)
        ax.add_patch(circle)
        for r in [3, 3.5, 4]:
            circle2 = Circle((0, 0), r, facecolor='none',
                        edgecolor='white', linewidth=1, alpha=1, zorder=1)
            ax.add_patch(circle2)
        im = NonUniformImage(ax, interpolation=interp, extent=(-1, 1, -1, 1))
        im.set_data(xcenters, ycenters, dist_mon_hpatch_t)
        im.set(clip_path=circle, clip_on=True)
        ax.add_image(im)
        #ax.axvline(x=3.5,ymin=0.25,ymax=0.75)
        #ax.axvline(x=-3.5,ymin=0.25,ymax=0.75)
        ax.set_box_aspect(1)
        #ax.set_xlim(-1.05, 1.05)
        ax.set_xticks(np.arange(-4.5,4.75,1))
        #ax.set_ylim(-1.05, 1.05)
        ax.set_yticks(np.arange(-4.5,4.75,1))
        ax.set_xlabel(r"$2x/D$")
        ax.set_ylabel(r"$2y/D$")
        ax.set_title(var[:-4] + r": $\rho(x,y)$")
        plt.savefig(filename +'-'+ var[:-3]+'pdf')
        plt.show()

# Test measurements:

In [ ]:
topo = '/Users/amirhsi_mini/research_data/HnsCyl-trjs/N200kbmm2r4.5nh30ac2lz75nc221ens2.ring/N200kbmm2r4.5nh30ac2lz75nc221ens2.ring.nucleoid.data'
traj = ['/Users/amirhsi_mini/research_data/HnsCyl-trjs/N200kbmm2r4.5nh30ac2lz75nc221ens2.ring/N200kbmm2r4.5nh30ac2lz75nc221ens2.ring.nucleoid.lammpstrj']
## 14 min for 2 trjs
## 9 min for N=200 no crowd no hns with all transformations, ndump=2000
## 9 min for N=200 no crowd no hns no need for all transformations, ndump=5000
path_txt = '/Users/amirhsi_mini/research_data/N1000_tests/N1000*/N*'
pairs =  sorted(glob(path_txt))
pairs = sort_filenames(pairs, ['j02.ring.dnaMinimize.nucleoid.lammpstrj', 'nucleoid.data'])

In [ ]:
%%time
topo = '/Users/amirhsi_mini/research_data/HnsCyl-trjs/N200kbmm2r4.5nh30ac2lz75nc221ens2.ring/N200kbmm2r4.5nh30ac2lz75nc221ens2.ring.nucleoid.data'
traj = ['/Users/amirhsi_mini/research_data/HnsCyl-trjs/N200kbmm2r4.5nh30ac2lz75nc221ens2.ring/N200kbmm2r4.5nh30ac2lz75nc221ens2.ring.nucleoid.lammpstrj']
## 14 min for 2 trjs
## 9 min for N=200 no crowd no hns with all transformations, ndump=2000
## 9 min for N=200 no crowd no hns no need for all transformations, ndump=5000
path_txt = '/Users/amirhsi_mini/research_data/N1000_tests/N1000*/N*'
pairs =  sorted(glob(path_txt))
pairs = sort_filenames(pairs, ['j02.ring.dnaMinimize.nucleoid.lammpstrj', 'nucleoid.data'])
for (trj, topo) in pairs:
    for i, trj in enumerate(traj):
        sim_info= HnsCyl(
            trj,
            'segment',
            'cylindrical',
            'nucleoid',
            'ring'
        )
        cell = mda.Universe(
            topo,
            trj,
            topology_format='DATA',
            format='LAMMPSDUMP',
            lammps_coordinate_convention='unscaled',
            unwrap_images=False,
            atom_style="id resid type x y z", 
        )
        if i == 0:
            sliced_trj = cell.trajectory
        else:
            sliced_trj = cell.trajectory
        n_frames = len(sliced_trj)
        bug = cell.select_atoms('resid 1')  # the bug
        #workflow = [
        #    mda_trans.unwrap(cell.atoms),
        #    mda_trans.center_in_box(bug),
        #    mda_trans.wrap(cell.atoms)
        #]
        #cell.trajectory.add_transformations(*workflow)
        # selecting atom groups:
        bug = cell.select_atoms('resid 1')  # monomers
        hns_patch = cell.select_atoms('type 2')  # the hns patches
        hns_core = cell.select_atoms('type 3')  # the hns patches
        # defining collectors
        # bug:
        gyr_t = []
        fsd_t = []
        trans_size_t = []
        principal_axes_t = []
        asphericity_t = []
        shape_parameter_t = []
        # - bond info
        #n_bonds = len(bug.bonds.indices)
        #bond_lengths = np.zeros((n_bonds, 1), dtype=np.float64)
        #cosine_corrs = np.zeros(n_bonds, dtype=np.float64)
        # distance matrices
        dist_m_hpatch_t = []
        d_contact_m_hpatch_t = []
        lj_cut = 2**(1/6)
        r_cutoff = np.round(
            0.5 * lj_cut * (sim_info.dmon + sim_info.dhns_patch), 3
            )
        topology = sim_info.topology
        for _ in sliced_trj:
            dummy = mda_dist.distance_array(bug, hns_patch)#, box=cell.dimensions)
            dist_m_hpatch_t.append(dummy)
            d_contact_m_hpatch = clusters.find_direct_contacts(
                dummy, r_cutoff, inclusive=False
                )
            d_contact_m_hpatch_t.append(d_contact_m_hpatch)
            # dummy = mda_dist.distance_array(bug, hns_core, box=cell.dimensions)
            # dist_m_hcore_t.append(dummy)
            # dummy = mda_dist.distance_array(bug, bug, box=cell.dimensions)
            # dist_m_m_t.append(dummy)
        dist_m_hpatch_t = np.array(dist_m_hpatch_t, dtype=float)
        d_contact_m_hpatch_t = np.array(d_contact_m_hpatch_t, dtype=int)
        # Saving collectors to memory
        # bug
        # Simulation stamps:
        # bond info
        #bond_lengths = bond_lengths / n_frames
        #bonds_per_lag = np.arange(n_bonds, 0, -1)
        #cosine_corrs = cosine_corrs / (n_frames * bonds_per_lag)
        #bond_lengths = bond_lengths.reshape(n_bonds,)
        print('done.')

### testing hns binding statisitcs (do not delete)

In [ ]:
results = {
        'n_m_hpatch_bound': [],
        'n_hpatch_free': [],
        'n_hpatch_engaged': [],
        'n_hcore_free': [],
        'n_hcore_bridge': [],
        'n_hcore_dangle': [],
        'n_mon_bound': [],
        'n_mon_cis': [],
        'n_mon_trans': []
    }
loop_length_hist = np.zeros((sim_info.nmon//2)+1, dtype=int)

direct_contact = d_contact_m_hpatch_t[100]

n_mon, n_hpatch = direct_contact.shape
n_hcore = n_hpatch // 2
# For both, ring with odd or even number of monomers, the maximum
# intra-chain distance is the same.
results['n_m_hpatch_bound'].append(np.sum(direct_contact))
d_cont_per_hpatch = np.sum(direct_contact, axis=0)
results['n_hpatch_engaged'].append(np.count_nonzero(d_cont_per_hpatch))
results['n_hpatch_free'].append(n_hpatch - results['n_hpatch_engaged'][-1])
results['n_hcore_free'].append(np.sum(
    (d_cont_per_hpatch[0::2] == 0) & (d_cont_per_hpatch[1::2] == 0)
    ))
results['n_hcore_bridge'].append(
    np.sum((d_cont_per_hpatch[0::2] > 0) & (d_cont_per_hpatch[1::2] > 0)
           ))
results['n_hcore_dangle'].append(
    n_hcore-results['n_hcore_free'][-1] - results['n_hcore_bridge'][-1]
    )
print(direct_contact.sum(axis=0))


enfoced_single_patch_direct_contact = direct_contact.copy()
# loop over columns
for j in range(enfoced_single_patch_direct_contact.shape[1]):
    # find index of first contact in column j
    i = np.argmax(enfoced_single_patch_direct_contact[:, j] != 0)
    # set values of elements after first contact to 0
    enfoced_single_patch_direct_contact[i+1:, j] = 0

cont_m_hpatch = clusters.generate_contact_matrix(enfoced_single_patch_direct_contact)
print(cont_m_hpatch.sum(axis=0))
cont_m_hcore = np.zeros((n_mon, n_hcore))
# for i in range(n_hcore):
#    cont_m_hcore[:,i] = cont_m_hpatch[:, 2*i] | cont_m_hpatch[:, 2*i+1]
cont_m_hcore= np.logical_or(cont_m_hpatch[:, ::2], cont_m_hpatch[:, 1::2])
print(cont_m_hcore.sum(axis=0))

cont_m_m = np.matmul(cont_m_hcore, cont_m_hcore.T)  # symmetric
cont_m_m_triu = np.triu(cont_m_m, 1)
cont_m_m_nonzeros = np.array(np.where(cont_m_m_triu > 0)).T
print(cont_m_m_nonzeros.shape)
results['n_mon_bound'].append(cont_m_m_nonzeros.shape[0])
m_m_gen_dist = clusters.hns_genomic_distance(cont_m_m_nonzeros, topology, n_mon)
print(m_m_gen_dist)
results['n_mon_cis'].append(np.count_nonzero(
    (m_m_gen_dist[:, 2] > 0) & (m_m_gen_dist[:, 2] <= 2)
    ))
results['n_mon_trans'].append(np.count_nonzero(m_m_gen_dist[:, 2] > 2))
np.add.at(loop_length_hist, m_m_gen_dist[:, 2], 1)

print(results)
print(loop_length_hist)

In [ ]:
# example contact matrix
contact_matrix = np.array([[1, 0, 0, 0], [0, 1, 1, 0], [1, 0, 1, 0], [0, 0, 1, 0]])
print(contact_matrix)
# loop over columns
for j in range(contact_matrix.shape[1]):
    # find index of first contact in column j
    print(contact_matrix[:, j])
    print(contact_matrix[:, j] != 0)
    i = np.argmax(contact_matrix[:, j] != 0)
    print(i)
    # set values of elements after first contact to 0
    contact_matrix[i+1:, j] = 0

print(contact_matrix)

In [ ]:
contact_matrix.sum(axis=1)

In [ ]:
direct_contact.sum(axis=0)

In [ ]:
cont_m_hpatch.sum(axis=0)

In [ ]:
n_hns = 12
hns = cell.select_atoms(f'resid 2:{n_hns+1}')
hhs_core = hns.select_atoms('type 2')
hhs_core.atoms.resids

In [ ]:
dist_mat_foci = np.zeros((hns_patch.n_atoms, hns_patch.n_atoms))
dist_array = mda_dist.self_distance_array(hns_patch)
k=0
for i in range(hns_patch.n_atoms):
    for j in range(i + 1, hns_patch.n_atoms):
        dist_mat_foci[i, j] = dist_array[k]
        k += 1

In [ ]:
mat_m_hpatch = np.array(
    [[1, 0, 1, 0, 0, 0, 0, 0],
     [1, 0, 0, 0, 0, 0, 1, 0],
     [1, 0, 0, 0, 0, 0, 1, 0],
     [0, 0, 0, 0, 0, 0, 0, 0],
     [0, 0, 0, 0, 0, 0, 0, 0],
     [0, 0, 0, 0, 0, 0, 0, 0],
     [0, 0, 0, 1, 0, 0, 0, 1],
     [0, 1, 0, 1, 0, 1, 0, 0],
     [0, 1, 0, 0, 0, 1, 0, 0],
     [0, 0, 0, 0, 0, 1, 0, 0]],
    dtype=int
)
print(" ", np.arange(1,9))
print(mat_m_hpatch)
print(np.arange(1,11))
print(mat_m_hpatch.T)
print()
mat_m_m = np.matmul(mat_m_hpatch,mat_m_hpatch.T)
print(mat_m_m)

In [ ]:
_, n_cols = mat_m_hpatch.shape  # shape: n_atoms * n_atoms
contact_matrix = mat_m_hpatch.copy()
# Loop over all pairs of column indices
for i, j in itertools.combinations(range(n_cols), 2):
    # Check if columns i and j have any common nonzero elements
    if np.any(contact_matrix[:, i] & contact_matrix[:, j]):
        # If so, set all nonzero elements in columns i and j to 1
        transient = contact_matrix[:, i] | contact_matrix[:, j]
        contact_matrix[:, i] = transient
        contact_matrix[:, j] = transient

contact_matrix

In [ ]:
mat_m_hcore = np.zeros((mat_m_hpatch.shape[0], mat_m_hpatch.shape[1]//2))
print(mat_m_hcore)
for i in range(n_cols//2):
    print(2*i)
    print(2*i+1)
    mat_m_hcore[:,i] = contact_matrix[:, 2*i] | contact_matrix[:, 2*i+1]

In [ ]:
mat_m_hcore

In [ ]:
import itertools
for i, j in itertools.product(range(5), range(4)):
    print(i,j)
print()
for i, j in itertools.combinations(range(5),2):
    print(i,j)

In [ ]:
mda_dist.self_distance_array(hns_patch)#, box=cell.dimensions)

In [ ]:
print(cell)
print(hasattr(cell, 'trajectory'))
print(len(cell.trajectory))
print(cell.atoms)
print(cell.residues)
print('bug')
print(bug.n_residues)
print(bug.residues)
print(bug.atoms.indices)
print(bug.positions)
print(bug.bonds.indices)
print('hns_patch')
print(hns_patch.n_residues)
print(hns_patch.residues)
print(hns_patch.atoms)
print('hns_core')
print(hns_core.n_residues)

# Viz different properties:

## Radius of gyration and farthermost distance plot

In [ ]:
color_palette = 'tab10'
plot_context = 'paper'
font_scale = 1.5
height = 5
aspect = 1.618
ylabel_pad = 50
rc_params= {
    #'axes.facecolor': 'aliceblue',
    'mathtext.default': 'regular',
    'text.usetex': True,
    #'axes.grid': True,
    #'axes.grid.axis': 'both',
    #'axes.grid.which': 'both'
}
fig, axes = plt.subplots(nrows=1, ncols=1,figsize=(aspect*height, height), sharex=True)
sns.set_theme(
    context=plot_context,
    style='ticks',
    palette=color_palette,
    font='Times New Roman',
    font_scale=font_scale,
    rc=rc_params
)
slicer = 0
ax = axes
ax.plot(time, gyr_t)
short_mean = np.mean(gyr_t[slicer:])
ax.axhline(y=short_mean,c='red',label=fr'$\langle R_g\rangle={round(short_mean,3)}$')
#ax.axvline(x=time[slicer],c='gray',ls='--',label=fr'$t/\tau_{{LJ}}={time[slicer]}$')
ax.legend(loc='lower right')
ax.set_ylabel(r'$R_g(t)$')
ax.set_title(r'Cylindrical confinement: $N=1000, D/a_m=15, a_c/a_m=1, \phi_c=0, n_{{hns}}=0$')
#ax = axes[1]
#fsd_mean = np.mean(fsd_t[slicer:])
#ax.plot(time, fsd_t)
#ax.axhline(y=fsd_mean,c='red',label=fr'$\langle L\rangle={round(fsd_mean,3)}$')
#ax.axvline(x=time[slicer],c='gray',ls='--',label=fr'$t/\tau_{{LJ}}={time[slicer]}$')
#ax.set_ylabel(r'$L$')
#ax.legend(loc='lower right')
ax.set_xlabel(r'$t/\tau_{LJ}$')
plt.show()
fig.savefig('chainsize.pdf')

## Persistance length

In [ ]:
#trj_cosine_corrs = np.mean(trj_cosine_corrs_mean,axis=0)
mean_bond = np.mean(trj_bond_lengths_mean[0])
print('bond length: ', mean_bond)
bond_lags = np.arange(1, n_bonds+1)
per_to_bond = fit_exponential_decay(
    bond_lags[:],
    trj_cosine_corrs_mean[0][:]
)
print('persisten to bond ratio', per_to_bond)
print('persisten length', per_to_bond*mean_bond)
print('persisten length in nm', per_to_bond*mean_bond*2.5)

In [ ]:
color_palette = 'tab10'
#color_palette = ['orange', 'darkgoldenrod', 'forestgreen']
save_to = './'
ext = 'pdf'
plot_context = 'paper'
font_scale = 1.5
height = 4
aspect = 1.5*1.618
ylabel_pad = 50
rc_params= {
    #'axes.facecolor': 'aliceblue',
    'mathtext.default': 'regular',
    'text.usetex': True,
    #'axes.grid': True,
    #'axes.grid.axis': 'both',
    #'axes.grid.which': 'both'
}
facet_kws = {
    'sharey': False,
    'sharex': False,
    'legend_out': True,
}
fig_title_kws = {'fontsize': 34, 'x': 0.5, 'y': 0.93}
loc='lower left'
font_family = 'sans-serif'
move_legend_kws = {
    #'ncol': 1,
    #'bbox_to_anchor': (1., 0.95),
    'frameon': False,
    #'facecolor': 'aliceblue',
    #'borderpad': 0.1,
    'markerscale': 1
}
fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(aspect*height, height))
sns.set_theme(
    context=plot_context,
    style='ticks',
    palette=color_palette,
    font='Times New Roman',
    font_scale=font_scale,
    rc=rc_params
)

ax.scatter(bond_lags,trj_cosine_corrs_mean[0],c='green',marker='o',s=10,alpha=0.7)
ax.plot(bond_lags,np.exp(-(bond_lags-1)/per_to_bond),color='red',label=fr'$\exp[-j/(l_p/l_b)]$ with $l_p/l_b={round(per_to_bond,3)}$')# (fitted to $75$ closest bonds)')
#short_mean = np.mean(long[55000:,1])
ax.axhline(y=0,c='black', lw=0.75)
#ax.axvline(x=0.005*long[55000,0],c='gray',ls='--',label='Equilibrium reached!')
ax.legend()
#ax.set_ylabel(r'$R_g(t)$')
ax.set_xlabel('Bond index lag, j')
#bond_grid.set_ylabels(r'$\langle\cos(\theta_{i,i+j})\rangle$')
ax.set_ylabel(r'$C(j)=\langle\hat{l}_i.\hat{l}_{i+j}\rangle$')
#ax.text(120, 0.7, fr'fit to first $25$ bonds')
ax.set_title(r'Cylindrical confinement: $N=200, D/a_m=15, a_c/a_m=1, \phi_c=0, n_{{hns}}=0$')
plt.show()
fig.savefig('bond_correlations.pdf')

### Non-linear fit

In [ ]:
def fit_exponential_non_linear_decay(x, y):
    r"""Fit a function to an exponential decay

    .. math::  y = \exp\left(- \frac{x}{a}\right)

    Parameters
    ----------
    x, y : array_like
      The two arrays of data

    Returns
    -------
    a : float
      The coefficient *a* for this decay

    Notes
    -----
    This function assumes that data starts at 1.0 and decays to 0.0

    """
    def exp_func(x, a, exponent):
        return np.exp(-1*x**exponent/a)
    a, exponent = scipy.optimize.curve_fit(exp_func, x, y)[0]
    return a, exponent

In [ ]:
gyr = np.concatenate((gyr1,gyr2),axis=0)
gyr.shape

In [ ]:
plot_context = 'paper'
font_scale = 2
height = 6
aspect = 1.618
rc_params= {
    #'axes.facecolor': 'aliceblue',
    'mathtext.default': 'regular',
    'text.usetex': True,
    #'axes.grid': True,
    #'axes.grid.axis': 'both',
    #'axes.grid.which': 'both'
}
fig, axes = plt.subplots(nrows=1, ncols=1,figsize=(aspect*height, height), sharex=True)
sns.set_theme(
    context=plot_context,
    style='ticks',
    palette=color_palette,
    font='Times New Roman',
    font_scale=font_scale,
    rc=rc_params
)



gyr=gyr_t
acf_t = tsas.acf(
            gyr,
            nlags=30000,
            alpha=0.05,
        )
acf = acf_t[0]
acf_lag = np.arange(1,len(acf)+1)
lower_cl = acf_t[1][:, 0] - acf_t[0]
upper_cl = acf_t[1][:, 1] - acf_t[0]
decay_time = fit_exponential_decay(
    acf_lag,
    acf
)
ax = axes
ax.plot(acf_lag, acf)
#ax.plot(acf_lag,np.exp(-1*acf_lag**decay_time[1]/decay_time[0]))
ax.plot(acf_lag,np.exp(-1*acf_lag/decay_time),label=fr'$\exp[-t/\tau]$ with $\tau={round(decay_time,3)}$')
ax.axhline(y=0,c='black', lw=0.75)
ax.fill_between(
            acf_lag,
            lower_cl,
            upper_cl,
            color='green',
            alpha=0.25,
            label=r'CI ($\alpha=0.05$)'
        )
ax.legend(ncols=2)
ax.set_title(r'Free space: $N=1000, D/a_m=15, a_c/a_m=1, \phi_c=0, n_{{hns}}=0$')
#ax.set_xlabel(r'Time lag, $t^{\prime}$')
ax.set_ylabel(r'$C_{R_g}(t^{\prime})$')


acf_t = tsas.acf(
            fsd_t,
            nlags=30000,
            alpha=0.05,
        )
acf = acf_t[0]
acf_lag = np.arange(1,len(acf)+1)
lower_cl = acf_t[1][:, 0] - acf_t[0]
upper_cl = acf_t[1][:, 1] - acf_t[0]
decay_time = fit_exponential_decay(
    acf_lag,
    acf
)
ax = axes[1]
ax.plot(acf_lag, acf)
ax.plot(acf_lag,np.exp(-1*acf_lag**decay_time[1]/decay_time[0]))
ax.plot(acf_lag,np.exp(-1*acf_lag/decay_time),label=fr'$\exp[-t/\tau]$ with $\tau={round(decay_time,3)}$')
ax.fill_between(
            acf_lag,
            lower_cl,
            upper_cl,
            color='green',
            alpha=0.25,
            label=r'CI ($\alpha=0.05$)'
        )
ax.legend(ncols=2)
ax.axhline(y=0,c='black', lw=0.75)
ax.set_xlabel(r'Time lag, $t$')
ax.set_ylabel(r'$C_{L}(t^{\prime})$')
fig.tight_layout()
fig.savefig('acf.pdf')
plt.show()

# Gyr and persistance length

In [ ]:
gyr=gyr_t

In [ ]:
plot_context = 'paper'
font_scale = 1.5
height = 5
aspect = 1.618
rc_params= {
    #'axes.facecolor': 'aliceblue',
    'mathtext.default': 'regular',
    'text.usetex': True,
    #'axes.grid': True,
    #'axes.grid.axis': 'both',
    #'axes.grid.which': 'both'
}
fig, axes = plt.subplots(nrows=2, ncols=1,figsize=(aspect*height, height))
sns.set_theme(
    context=plot_context,
    style='ticks',
    palette=color_palette,
    font='Times New Roman',
    font_scale=font_scale,
    rc=rc_params
)

ax = axes[0]
idx_slice = 0
ax.plot(time,gyr)
short_mean = np.mean(gyr[idx_slice:])
ax.axhline(y=short_mean,c='red',label=fr'$\langle R_g\rangle={round(short_mean,3)}$')
#ax.axvline(x=time[idx_slice],c='gray',ls='--',label=fr'$t/\tau_{{LJ}}={time[idx_slice]}$')
ax.legend()
ax.set_ylabel(r'$R_g(t)$')
ax.set_xlabel(r'$t/\tau_{LJ}$')
ax.set_title(r'Free space: $N=1000, D/a_m=15, a_c/a_m=1, \phi_c=0, n_{{hns}}=0$')

ax = axes[1]
ax.scatter(bond_lags,trj_cosine_corrs_mean[0],marker='o',s=5,alpha=0.7)
ax.plot(bond_lags,np.exp(-(bond_lags-1)/per_to_bond),color='red',label=fr'$\exp[-j/(l_p/l_b)]$ with $l_p/l_b={round(per_to_bond,3)}$')
#short_mean = np.mean(long[55000:,1])
ax.axhline(y=0,c='black', lw=0.75)
ax.legend()
#ax.set_title(fr'Bond-bond ACF $C(j)$ for $t/\tau_{{LJ}}\ge{0.005*gyr[55019,0]}$')
ax.set_xlabel('Bond index lag, j')
ax.set_ylabel(r'$C(j)=\langle\hat{l}_i.\hat{l}_{i+j}\rangle$')
#ax.text(120, 0.7, fr'$l_p/l_b={round(per_length,3)}$')
fig.tight_layout()
plt.show()
fig.savefig('gyr-bond_correlations.pdf')